In [1]:
!pip install TTS

     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------- ----------------------------- 0.5/1.7 MB 9.6 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 21.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for cython>=0.29.30 from https://files.pythonhosted.org/packages/85/7d/58635941dfbb5b4e197adb88080b9cbfb230dc3b75683698a530a1989bdb/Cython-3.0.8-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.11.2 from https://files.pythonhosted.org/packages/9a/25/5b30cb3efc9566f0ebeaeca1976150316353c17031ad7868ef46de5ab8dc/scipy-1.12.0-cp311-cp311-win_amd64.whl.metadata
     ---------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.
s3fs 2023.3.0 requires fsspec==2023.3.0, but you have fsspec 2023.12.2 which is incompatible.


In [2]:
from TTS.api import TTS

In [7]:
!pip install -r requirements.txt

  Obtaining dependency information for python-pptx==0.6.22 from https://files.pythonhosted.org/packages/c0/ea/94b20a70ed151c4af20dbc3aaa275a89757964f9df5a960698e399012c5e/python_pptx-0.6.22-py3-none-any.whl.metadata
  Obtaining dependency information for PyMuPDF==1.23.16 from https://files.pythonhosted.org/packages/44/ca/22607b88bdc6b27ab39fc0cfe878310a35cddd5f671111cbc7398c1103fd/PyMuPDF-1.23.16-cp311-none-win_amd64.whl.metadata
     ---------------------------------------- 0.0/388.3 kB ? eta -:--:--
     ------------------------------------- 388.3/388.3 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for openai==1.8.0 from https://files.pythonhosted.org/packages/f1/d8/590a68d390501faf48f4e57b098076df02afd003ac880f50d3b0704f7773/openai-1.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for comtypes==1.2.1 from https://files.pythonhosted.org/packages/6e/15/b

In [5]:
tts_manager = TTS().list_models()
all_models = tts_manager.list_models()
print(all_models)

['tts_models/multilingual/multi-dataset/xtts_v2', 'tts_models/multilingual/multi-dataset/xtts_v1.1', 'tts_models/multilingual/multi-dataset/your_tts', 'tts_models/multilingual/multi-dataset/bark', 'tts_models/bg/cv/vits', 'tts_models/cs/cv/vits', 'tts_models/da/cv/vits', 'tts_models/et/cv/vits', 'tts_models/ga/cv/vits', 'tts_models/en/ek1/tacotron2', 'tts_models/en/ljspeech/tacotron2-DDC', 'tts_models/en/ljspeech/tacotron2-DDC_ph', 'tts_models/en/ljspeech/glow-tts', 'tts_models/en/ljspeech/speedy-speech', 'tts_models/en/ljspeech/tacotron2-DCA', 'tts_models/en/ljspeech/vits', 'tts_models/en/ljspeech/vits--neon', 'tts_models/en/ljspeech/fast_pitch', 'tts_models/en/ljspeech/overflow', 'tts_models/en/ljspeech/neural_hmm', 'tts_models/en/vctk/vits', 'tts_models/en/vctk/fast_pitch', 'tts_models/en/sam/tacotron-DDC', 'tts_models/en/blizzard2013/capacitron-t2-c50', 'tts_models/en/blizzard2013/capacitron-t2-c150_v2', 'tts_models/en/multi-dataset/tortoise-v2', 'tts_models/en/jenny/jenny', 'tts_m

In [11]:
from pptx import Presentation
from pptx.util import Inches
import subprocess
import fitz  # PyMuPDF
import os
from moviepy.editor import *
from pptx_to_pdf import convert_pptx_to_pdf
import re
from TTS.api import TTS


#importing API key etc...
import openai
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
openai_api = os.getenv('OPENAI_API')

if openai_api is None:
    raise ValueError("The OPEN_AI environment variable is not set.")
#don't change this part of the code. Should remain the same

#Set Global Variables
powerpoint_file_path = "my_presentation.pptx"   # Path to the PowerPoint file
pdf_file_path = "my_presentation.pdf"  # Path to the PDF file
images_folder = "slide_images"  # Path to the folder where the images will be stored
video_file_path = "my_presentation.mp4"  # Path to the output video file
frame_rate = 30  # Adjust the frame rate as needed
duration_per_image = 3  # Time each image is shown in seconds
transition_duration = 0.5  # Duration of the transition between images in seconds

template_file = 'template.pptx'#load the powerpoint template
prs = Presentation(template_file)

# text to speeech model
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2")

#generate chatgpt slides content LLM content generation (prompt argument is the topic of the presentation)
def create_content(prompt):
    print("Converting to educational content....")
    client = openai.OpenAI(api_key=openai_api)
    story = client.chat.completions.create(
        messages = [
        {"role": "system", "content": "Create content for a 20-slide presentation on the input topic. For each slide, provide a title and up to six bullet points that effectively cover different aspects of the topic. Ensure that the content is informative, accurate, and well-structured to facilitate a comprehensive understanding of the subject. Lable the title of each slide as Title: followed by the name of the title of the slide"},
        {"role": "user", "content": prompt}],
        model="gpt-3.5-turbo",
    )
    # write response to the file
    nextpass = story.choices[0].message.content  
    file_name = "chatgptresponse.txt"
    with open(file_name, 'w', encoding="utf-8") as file:
        # Write the string to the file
        file.write(nextpass)

def convert_script_to_audio(script: str, slide_number: int):
    # Convert the script to an MP3 file and save it
    audio_path = f"slide_audio/slide_{slide_number}.mp3"
    tts.tts_to_file(text=script, output_path =audio_path, language="en")


def add_title(titleName):
    slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    title.text = titleName

def addContent(titleName,contentBox):
    slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    title.text = titleName
    content = slide.placeholders[1]
    content.text = contentBox
 
def imageContent(imgpath, titleName, contentBox,slideType):
    slide_layout = prs.slide_layouts[slideType]
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    title.text = titleName
    content = slide.placeholders[1]
    content.text = contentBox
    image_placeholder = slide.placeholders[2]
    image_path = imgpath
    left = image_placeholder.left
    top = image_placeholder.top
    width = image_placeholder.width
    height = image_placeholder.height
    slide.shapes.add_picture(image_path, left, top, width, height)

#create presentation based on chatgpt text (stored in a text file)
def create_presentation_from_text(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    slides = re.split(r'Title:', content)[1:]
    convert_script_to_audio("Thanks for listening <3", len(slides) + 2)

    for (slide_number, slide) in enumerate(slides):
        convert_script_to_audio(slide, slide_number + 1 + 1) # + 1 for 1 indexing, +1 for title

        title_match = re.match(r'(.+)', slide)
        bullet_points_match = re.findall(r'- (.+)', slide)
        if title_match:
            title = title_match.group(1).strip()
            bullet_points = '\n'.join(bullet_points_match) 
            addContent(title, bullet_points)  
    add_title("Thanks for listening <3")
    prs.save(powerpoint_file_path)

def pdf_to_images(pdf_path, output_folder):
    pdf_document = fitz.open(pdf_path)
    
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        image = page.get_pixmap()
        
        image_path = f"{output_folder}/page_{page_num + 1}.png"
        image.save(image_path)

    pdf_document.close()

def create_clip(image_path, clip_duration):
    return ImageClip(image_path, duration=clip_duration)

def get_images(images_folder):
    # Get all the images path in the folder
    images = [img for img in os.listdir(images_folder) if img.endswith(".png")]

    def get_page_number(image_name):
        # Extract the numerical part of the filename
        return int(''.join(filter(str.isdigit, image_name)))

    images.sort(key=get_page_number)

    images = [os.path.join(images_folder, img) for img in images]

    return images

def create_slideshow(images_folder, output_video_path, frame_rate, duration_per_image, transition_duration):

    images = get_images(images_folder)

    image_clips = [create_clip(image, duration_per_image) for image in images]

    def fadeout(clip, duration):
        #return the portion of the clip when it is fading out
        fade = clip.subclip(clip.duration - duration, clip.duration)
        return CompositeVideoClip([fade.crossfadeout(duration)])
    
    def fadein(clip, duration):
        #return the portion of the clip when it is fading in
        fade = clip.subclip(0, duration)
        return CompositeVideoClip([fade.crossfadein(duration)])

    final_clip = []

    for (i, clip) in enumerate(image_clips):
        # add audio to clip
        audioclip = AudioFileClip(f"slide_audio/slide_{i + 1}.mp3")

        # add transition to clip
        fadein_clip = fadein(clip, transition_duration)
        fadeout_clip = fadeout(clip, transition_duration)
        final_clip.append(fadein_clip)
        clip = clip.set_audio(audioclip)
        final_clip.append(clip.set_duration(audioclip.duration))
        final_clip.append(fadeout_clip)

    concatenated_clip = concatenate_videoclips(final_clip)
    bgm = AudioFileClip("bgm/John_Bartmann_-_01_-_Mad_Hatter_Tea_Party(chosic.com).mp3").volumex(0.5)
    concatenated_clip.audio = CompositeAudioClip([concatenated_clip.audio, audioclip])

    concatenated_clip.write_videofile(output_video_path, codec='libx264', fps=frame_rate)

if __name__ == "__main__":
    topic = "cheese"
    # create_content(topic) #generate chatgpt script for presentation - parameter is topic
    #change prompt as you feel is right in order to get a script that will flow well 

    # create title audio clip
    convert_script_to_audio(topic, 1)
   
    file_path = "chatgptresponse.txt"
    add_title(topic)
    create_presentation_from_text(file_path)# create slides from script file 

    convert_pptx_to_pdf(powerpoint_file_path, pdf_file_path)

    pdf_to_images(pdf_file_path, images_folder)

    create_slideshow(images_folder, video_file_path, frame_rate, duration_per_image, transition_duration)

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
 > Text splitted to sentences.
['cheese']


LibsndfileError: Error opening 'my/cloning/audio.wav': System error.